In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import tensorflow as tf
print("Packages loaded!")

Packages loaded!


In [2]:
_data_path = "mnist/"

mnist = input_data.read_data_sets(_data_path, one_hot=True)
print("Data loaded!!")

Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz
Data loaded!!


In [3]:
img_test = mnist.test.images
label_test = mnist.test.labels

NUM_CLASSES = 10
IMAGE_SIZE = 28
IMAGE_PIXEL = IMAGE_SIZE * IMAGE_SIZE

BATCH_SIZE = 100
total_batch = int(mnist.train.num_examples / BATCH_SIZE)

HIDDEN1_UNITS = 128
HIDDEN2_UNITS = 32

MAX_EPOCH = 300

In [ ]:
def linear(data_in, output_units, name='name'):
    with tf.variable_scope(name):
        input_units = data_in.get_shape().as_list()[-1]
        weights = tf.get_variable('weights', shape=[input_units, output_units], initializer=tf.contrib.layers.xavier_initializer())
        biases = tf.Variable(tf.zeros([output_units]), name='biases')
        logits = tf.nn.xw_plus_b(data_in, weights, biases, name='logits')
        activation = tf.sigmoid(logits, name="act")
    return logits, activation


def inference_mlp1(data_in, num_hidden_node):
    _, h1 = linear(data_in, num_hidden_node, name="linear1")
    logits, _ = linear(h1, NUM_CLASSES, name="linear2")
    return logits


def inference_mlp2(data_in, num_hidden1_node, num_hidden2_node):
    _, h1 = linear(data_in, num_hidden1_node, name="linear1")
    _, h2 = linear(h1, num_hidden2_node, name="linear2")
    logits, _ = linear(h2, NUM_CLASSES, name="linear3")
    return logits


def loss(logits, labels):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, labels, name="x_entropy")
    loss = tf.reduce_mean(cross_entropy, name="loss")
    return loss


def training(loss, learning_rate):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    global_step = tf.Variable(0, name='global_step', trainable=False)
    train = optimizer.minimize(loss, global_step)
    return train


def evaluation(logits, labels):
    correct = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, "float"))
    return accuracy

In [ ]:
NUM_GPU = 2

mnist_graph = tf.Graph()

with mnist_graph.as_default(), tf.device('/cpu:0'):
    images_placeholder = []
    labels_placeholder = []
    for i in xrange(NUM_GPU) :
        images_placeholder.append(tf.placeholder(tf.float32, [None, IMAGE_PIXEL]))
        labels_placeholder.append(tf.placeholder(tf.float32, [None, NUM_CLASSES]))

    #labels_placeholder = tf.placeholder(tf.float32)
    tf.add_to_collection("images", images_placeholder)
    tf.add_to_collection("labels", labels_placeholder)
    
    global_step = tf.get_variable('global_step', [],
                                 initializer=tf.constant_initializer(0.001), trainable=False)
    
    optimizer = tf.train.AdamOptimizer(0.001)
    
    tower_loss = []
    tower_grads = []
    tower_accr = []
    
    for i in xrange(NUM_GPU) : 
        with ktf.device('/gpu:%d', %i) :
            with tf.name_scope('tower_%d %i') as scope :
                logits = inference_mlp1(image_placeholder[i[, HIDDEB1_UNITS]])
                tf.get_variable_scope().reuse_variables()
                cost = loss(logits, labless_placeholder),

    #logits = inference_mlp1(images_placeholder, HIDDEN1_UNITS)
    logits = inference_mlp2(images_placeholder, HIDDEN1_UNITS, HIDDEN2_UNITS)
    tf.add_to_collection("logits", logits)

    cost = loss(logits, labels_placeholder)
    train_op = training(cost, 0.001)

    init = tf.initialize_all_variables()

with tf.Session(graph=mnist_graph) as sess:
    sess.run(init)

    for step in xrange(MAX_EPOCH * BATCH_SIZE + 1):
        img_batch, label_batch = mnist.train.next_batch(BATCH_SIZE)
        feeds = {images_placeholder: img_batch, labels_placeholder: label_batch}
        loss_value = sess.run(cost, feeds)
        sess.run(train_op, feeds)

        if step % 500 == 0:
            print("Step %d: loss = %.4f" % (step, loss_value))
            eval_train = evaluation(logits, labels_placeholder)
            feeds_test = {images_placeholder: mnist.test.images, labels_placeholder: mnist.test.labels}
            eval_test = evaluation(logits, labels_placeholder)
            print("Train : %.2f%%, Test : %.2f%%" % (sess.run(eval_train, feeds)*100, sess.run(eval_test, feeds_test)*100))

print("Done!!")


Step 0: loss = 2.4273
Train : 12.00%, Test : 9.58%
Step 500: loss = 0.3651
Train : 92.00%, Test : 90.81%
Step 1000: loss = 0.2309
Train : 97.00%, Test : 93.16%
Step 1500: loss = 0.2897
Train : 92.00%, Test : 93.96%
Step 2000: loss = 0.2498
Train : 95.00%, Test : 94.93%
Step 2500: loss = 0.1320
Train : 95.00%, Test : 95.58%
Step 3000: loss = 0.1071
Train : 98.00%, Test : 96.23%
Step 3500: loss = 0.0239
Train : 100.00%, Test : 96.51%
Step 4000: loss = 0.0546
Train : 99.00%, Test : 96.95%
Step 4500: loss = 0.0402
Train : 99.00%, Test : 96.94%
Step 5000: loss = 0.0603
Train : 99.00%, Test : 97.28%
